In [63]:
import os
import uuid
import csv
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool
from dotenv import load_dotenv
load_dotenv()

True

In [64]:
# Initialize LLM
google_llm = LLM(
    model="gemini/gemini-2.0-flash-exp",
    temperature=0.7,
)

In [65]:
from dotenv import load_dotenv
load_dotenv()

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [66]:
from typing import List, Type
from pydantic import BaseModel, Field, PrivateAttr
from crewai.tools import BaseTool
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [67]:
class MDXSearchInput(BaseModel):
    query: str = Field(..., description="The search query to find relevant content in the MDX file.")

class MDXSearchTool(BaseTool):
    name: str = "Search an MDX's content"
    description: str = "A tool that performs semantic search over an MDX file's content."
    args_schema: Type[BaseModel] = MDXSearchInput
    _mdx_path: str = PrivateAttr()
    _documents: List[str] = PrivateAttr()
    _document_embeddings: np.ndarray = PrivateAttr()

    def __init__(self, mdx_path: str, **kwargs):
        super().__init__(**kwargs)
        self._mdx_path = mdx_path
        self._documents = self.load_mdx_content()
        self._document_embeddings = self.embed_documents(self._documents)

    def load_mdx_content(self) -> List[str]:
        with open(self._mdx_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content.split('\n\n')

    def embed_documents(self, documents: List[str]) -> np.ndarray:
        return embedding_model.encode(documents)

    def _run(self, query: str) -> str:
        query_embedding = embedding_model.encode([query])[0]
        similarities = np.dot(self._document_embeddings, query_embedding)
        most_similar_idx = np.argmax(similarities)
        return self._documents[most_similar_idx]

# Initialize the MDXSearchTool with the path to your MDX file
mdx_search_tool = MDXSearchTool(mdx_path=r'res.md')


In [68]:
# 2. Explicitly verify the API key
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
print(f"Serper API Key: {'Exists' if SERPER_API_KEY else 'Missing'}")  # Debug check

Serper API Key: Exists


In [69]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool(serper_api_key=os.getenv("SERPER_API_KEY"),
                            n_results=20)
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=r'C:\Users\Admin\Pictures\c\AGENT\Test_8\res.md')
# semantic_search_resume = MDXSearchTool(mdx='new.md')

In [70]:
from crewai_tools import WebsiteSearchTool

tool = WebsiteSearchTool(
    config={
        "embedder": {
            "provider": "huggingface",
            "config": {
                "model": "sentence-transformers/all-MiniLM-L6-v2"  # Model name as string
            }
        }
    }
)

In [82]:
# Test embedding dimensions match the tool's expectations
test_embedding = embedding_model.encode("test").tolist()
print(f"Embedding dimension: {len(test_embedding)}")  # Should output 384

Embedding dimension: 384


In [71]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool,tool, 
             search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [72]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for AI Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, tool,
             search_tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [73]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for AI Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, 
             search_tool,tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [74]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="AI Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, 
             search_tool, tool,
             read_resume, mdx_search_tool],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = google_llm
)

LLM value is already an LLM object


In [75]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [76]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [77]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [78]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [79]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
)

2025-01-29 17:02:05,891 - 11672 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [80]:
job_application_inputs = {
    'job_posting_url': 'https://in.linkedin.com/jobs/artificial-intelligence-jobs-gurugram?currentJobId=4082429484&position=1&pageNum=0',
    'github_url': 'https://github.com/antyoday94',
    'personal_writeup': """Antyoday is an accomplished AI
    Engineering Leader with 2 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [81]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://in.linkedin.com/jobs/artificial-intelligence-jobs-gurugram?currentJobId=4082429484&position=1&pageNum=0) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for AI Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/antyoday94) URLs, and personal write-up (Antyoday is an accomplished AI
    Engineering Leader with 2 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovati

2025-01-29 17:06:08,043 - 11672 - llm.py-llm:277 - ERROR: LiteLLM call failed: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



# Agent: AI Engineering Interview Preparer
## Thought: Thought: I've gathered some common interview questions for AI Engineering Leader roles. Now, I will craft specific questions and talking points tailored to Antyoday's resume and the job market context. This will include both technical and leadership-focused questions.
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"C:\\\\Users\\\\Admin\\\\Pictures\\\\c\\\\AGENT\\\\Test_8\\\\res.md\"}"
## Tool Output: 
# Abhay Verma

**Phone:** +91 6387401801  
**Email:** abhayverma6300@gmail.com

## Experience

**Celebal Technologies, Jaipur, RJ**  
*Associate Data Scientist*  
*June 2023 – Present*

- Developed and deployed an NLP pipeline for Named Entity Recognition (NER) using a fine-tuned transformer model, significantly enhancing document analysis and usability.
- Fine-tuned a BERT-based document-level relation extraction model for extr

In [46]:
result

CrewOutput(raw="```\n**Interview Questions and Talking Points**\n\n**For Antyoday Kumar**\n\n**General Background and Leadership:**\n\n1.  **Question:** Your resume highlights your experience as an AI Engineering Leader. Can you describe a situation where you had to manage a complex project with a remote team and how you ensured its success?\n    *   **Talking Points:**\n        *   Emphasize your experience in managing both remote and in-office teams.\n        *   Discuss specific strategies you used for communication, collaboration, and task management in a remote setting.\n        *   Highlight how you ensured the project met its goals and deadlines.\n        *   Mention tools or methodologies you used to foster team cohesion and productivity.\n2.  **Question:** The job description requires experience in promoting IT business solutions and executing a territory business strategy. How have you applied your strategic planning skills in your previous roles, and what were the outcomes?\

In [47]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Abhay Verma

**Phone:** +91 6387401801  
**Email:** abhayverma6300@gmail.com

**Summary**

Results-driven AI and Data Science professional with a proven track record of developing and deploying innovative AI solutions. Experienced in leveraging machine learning and deep learning techniques to solve complex business problems. Seeking to apply technical expertise and strong communication skills to drive sales and business growth within the Google Cloud AI ecosystem. Adept at understanding customer needs and translating them into impactful AI solutions, and capable of partnering with cross-functional teams to deliver exceptional results.

## Experience

**Celebal Technologies, Jaipur, RJ**  
*Associate Data Scientist*  
*June 2023 – Present*

- Developed and deployed an NLP pipeline for Named Entity Recognition (NER) using a fine-tuned transformer model, significantly enhancing document analysis and usability, demonstrating ability to create customer value.
- Fine-tuned a BERT-based document-level relation extraction model for extracting relations between named entities, showcasing ability to work with advanced AI models.
- Expanded transformer model context to 8k tokens using RoPE, enhancing long-document processing, highlighting skills in advanced model implementation.
- Engineered and deployed a Retrieval-Augmented Generation (RAG) chatbot for querying client policy documents across multiple formats, utilizing Azure Document Intelligence and AI Search, demonstrating experience with cloud-based AI services.
- Integrated Azure Computer Vision for video retrieval, facilitating direct linking to relevant video segments, displaying innovative application of AI technology.
- Managed session-wise conversation history with CosmosDB and automated file management via Azure Logic Apps, showing experience in cloud data management and workflow automation.
- Deployed containerized APIs on Azure App Service for seamless integration with web apps and Microsoft Teams, indicating proficiency in cloud deployment strategies.

**Celebal Technologies, Jaipur, RJ**  
*Data Science Intern*  
*Feb 2023 – May 2023*

- Developed a solar panel defect detection system using Detectron2, achieving 94% average precision for cell detection, streamlining maintenance operations, highlighting practical AI application.
- Implemented data augmentation strategies and optimized hyperparameters to enhance model performance, showcasing ability to improve AI model performance.
- Conducted exploratory data analysis (EDA) and optimized prediction models for airport manpower planning, demonstrating analytical skills and ability to drive insights.
- Gained hands-on experience with Azure AI services, focusing on NLP and RAG pipelines, proving familiarity with cloud AI platforms.

## Projects

**Medical Document Analysis**

- Developed an NLP pipeline for medical document analysis, achieving 86% F1 score in Named Entity Recognition (NER) and 75% F1 score in document-level named entity relation extraction. Deployed solution via FastAPI with entity highlighting, demonstrating end-to-end AI project delivery.

**RAG Chatbot Development**

- Developed a RAG chatbot on 15,000 pages from HTML, PDFs, DOCX, and Excel files. Conducted knowledge enrichment, increasing retrieval accuracy to 95%, and engineered prompts for robust and accurate answering, showcasing expertise in information retrieval and NLP.

## Skills

- **AI & Machine Learning:** Generative AI, Natural Language Processing (NLP), Deep Learning, Fine-Tuning,  Machine Learning Algorithms, Predictive Modeling, Data Mining
- **Cloud Computing:** Azure AI Services, Azure App Service, Azure Document Intelligence, Azure AI Search, Azure Computer Vision, CosmosDB, Azure Logic Apps, Cloud Deployment
- **Programming:** Python, SQL
- **Business Acumen:**  Customer Value Proposition, Solution Use Case Identification, Business Growth Strategies, Stakeholder Engagement, Strategic Account Planning, Go-to-Market Strategies
- **Communication & Collaboration:** Excellent communication skills, Cross-functional team collaboration, Client Relationship Management

## Education

**Galgotias University, Greater Noida, UP**  
*B.Tech. in Computer Science & Engineering (specialization in AI & ML)*  
*June 2023*

**Sunbeam Suncity School, Varanasi, UP**  
*Senior Secondary*  
*June 2018*

## Certifications

- Microsoft Certified: Azure Fundamentals | Azure AI Fundamentals
- Coursera Specializations: Neural Networks and Deep Learning, Improving Deep Neural Networks: Hyperparameter Tuning, Regularization, and Optimization, Convolutional Neural Networks, Sequence Models
- Deep Learning – IIT Ropar

## Additional Information

- GitHub: [https://github.com/abhayverma6300](https://github.com/abhayverma6300)
- LinkedIn: [https://www.linkedin.com/in/abhay-verma-944b84224/](https://www.linkedin.com/in/abhay-verma-944b84224/)
- Kaggle: [https://www.kaggle.com/abhayverma6300](https://www.kaggle.com/abhayverma6300)

In [48]:
from IPython.display import Markdown, display
display(Markdown("./interview_materials.md"))

```
**Interview Questions and Talking Points**

**For Antyoday Kumar**

**General Background and Leadership:**

1.  **Question:** Your resume highlights your experience as an AI Engineering Leader. Can you describe a situation where you had to manage a complex project with a remote team and how you ensured its success?
    *   **Talking Points:**
        *   Emphasize your experience in managing both remote and in-office teams.
        *   Discuss specific strategies you used for communication, collaboration, and task management in a remote setting.
        *   Highlight how you ensured the project met its goals and deadlines.
        *   Mention tools or methodologies you used to foster team cohesion and productivity.
2.  **Question:** The job description requires experience in promoting IT business solutions and executing a territory business strategy. How have you applied your strategic planning skills in your previous roles, and what were the outcomes?
    *   **Talking Points:**
        *   Share an example of a strategic plan you developed and implemented.
        *   Explain how your plan contributed to the growth or success of your team or organization.
        *   Focus on measurable outcomes and the impact of your strategic decisions.
        *   Connect your experience to the sales and business growth focus of the role.
3.  **Question:** You have an MBA and a strong background in AI and data science. How do you see the intersection of these two fields in the context of driving business growth?
    *   **Talking Points:**
        *   Explain how your business acumen complements your technical skills.
        *   Discuss how you can translate complex technical concepts into business value.
        *   Provide examples of how you have used data-driven insights to inform business decisions.
        *   Position yourself as someone who understands both the technical and business sides of AI solutions.

**Technical and AI Expertise:**

4.  **Question:** The role involves promoting AI technology. Can you explain a complex AI concept to someone with a non-technical background, and how would you tailor your explanation to different stakeholders?
    *   **Talking Points:**
        *   Choose a specific AI concept (e.g., machine learning, deep learning, NLP).
        *   Explain it in simple terms, avoiding technical jargon.
        *   Discuss how you would adapt your explanation for a CEO versus a technical lead.
        *   Show your ability to communicate effectively with diverse audiences.
5.  **Question:** The role requires experience in identifying and driving use cases to solve customer problems. Can you describe a situation where you identified a critical business problem and proposed an AI-driven solution?
    *   **Talking Points:**
        *   Describe a specific business problem you encountered.
        *   Explain how you identified the problem and gathered relevant data.
        *   Detail the AI solution you proposed and how it addresses the problem.
        *   Highlight the impact and value of your solution.
6.  **Question:** While your resume mentions expertise in multiple programming languages and frameworks, it doesn't list specifics. Can you share some of the key languages and frameworks you are proficient in and how they've helped in your projects?
    *   **Talking Points:**
        *   List specific programming languages (e.g., Python, Java, etc.) and frameworks (e.g., TensorFlow, PyTorch, etc.) you know well.
        *   Provide examples of how you've used these tools in your projects.
        *   Show your hands-on experience and practical application of these technologies.

**Sales and Business Acumen:**

7.  **Question:** The role requires building relationships with new and existing customers and internal sales teams. How would you approach building these relationships and ensuring you're a trusted advisor?
    *   **Talking Points:**
        *   Share your strategies for building trust and rapport.
        *   Discuss your approach to understanding customer needs and pain points.
        *   Explain how you would collaborate with internal sales teams to drive business growth.
        *   Highlight your ability to act as a subject matter expert and trusted advisor.

**For Abhay Verma**

**Technical and AI Expertise:**

1.  **Question:** You have hands-on experience with various AI technologies, including NLP and RAG. Can you describe a complex technical challenge you faced while developing your RAG chatbot and how you overcame it?
    *   **Talking Points:**
        *   Describe the specific challenge you faced (e.g., data quality, retrieval accuracy).
        *   Explain the steps you took to address the challenge.
        *   Show your problem-solving skills and technical depth.
        *   Highlight your ability to work with advanced AI techniques.
2.  **Question:** You've worked with Azure AI services. How would you compare Azure's AI offerings with Google Cloud's AI offerings, and what would you highlight as unique selling points for Google Cloud AI?
    *   **Talking Points:**
        *   Demonstrate your familiarity with both Azure and Google Cloud AI services.
        *   Compare their strengths and weaknesses.
        *   Highlight specific Google Cloud AI features that provide a competitive advantage.
        *   Show your understanding of the market landscape and your ability to position Google Cloud effectively.
3.  **Question:** Your projects demonstrate strong NLP skills. How would you apply your NLP expertise to solve a business problem for a potential client in the retail or healthcare industry?
    *   **Talking Points:**
        *   Choose a specific business problem in either retail or healthcare (e.g., customer sentiment analysis, medical document processing).
        *   Explain how you would use NLP to address the problem.
        *   Focus on the business value and impact of your proposed solution.
        *   Show your ability to translate technical skills into practical business applications.
4.  **Question:** You’ve worked with fine-tuning transformer models. Can you explain the importance of fine-tuning and how it contributes to the overall performance of AI models?
    *   **Talking Points:**
        *   Explain the concept of fine-tuning in simple terms.
        *   Discuss why it's important for model performance and accuracy.
        *   Share examples of how fine-tuning improved your models.
        *   Show your understanding of AI model optimization techniques.
5.  **Question:** You used Azure Document Intelligence for the RAG Chatbot. What are the key features of document intelligence that are important for this use case?
    *   **Talking Points:**
        *   Identify key features of document intelligence like OCR, form recognition, and table extraction.
        *   Explain how these features were used for the RAG Chatbot.
        *   Demonstrate an understanding of the tools and services used in the project.

**Sales and Business Acumen:**

6.  **Question:** This role requires you to connect AI/ML with business value. Can you provide an example of how you’ve translated a technical AI solution into a compelling business value proposition?
    *   **Talking Points:**
        *   Share an example where you explained the business impact of an AI solution.
        *   Focus on the benefits, ROI, and strategic value for the client.
        *   Show your ability to articulate value propositions to non-technical stakeholders.
        *   Highlight your understanding of the customer's perspective.

7.  **Question:** The job description emphasizes driving business growth. How would you approach identifying new business opportunities and expanding existing customer relationships within the AI domain?
    *   **Talking Points:**
        *   Explain your strategies for identifying new market opportunities.
        *   Discuss your approach to nurturing existing client relationships.
        *   Focus on your ability to understand customer needs and develop tailored solutions.
        *   Highlight your proactive approach to driving business growth.
8. **Question:** How would you handle a situation where a potential client is hesitant about adopting AI due to concerns about complexity or cost?
    *   **Talking Points:**
        *   Outline your approach to addressing client concerns.
        *   Explain how you would break down complex concepts into simpler terms.
        *   Discuss your strategy for demonstrating ROI and value to the client.
        *   Show your ability to handle objections and build trust.

These questions and talking points are designed to help both candidates showcase their strengths and align their experiences with the requirements of the AI Sales Specialist role at Google Cloud.
```